## Nodemcu ESP32 重建 Micropython Kernel

In [33]:
%esptool --port COM4 erase

Executing:
  esptool --port COM4 erase_flash

esptool.py v2.8
Serial port COM4
Connecting........___
[Press the PRG button now if required]
Detecting chip type... ESP32
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: cc:50:e3:80:1f:20
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 2.9s
Hard resetting via RTS pin...


In [34]:
%esptool --port com4 esp32 esp32-idf3-20191209-v1.11-624-g210d05328.bin

Executing:
  esptool --port com4 --chip esp32 write_flash -z 0x1000 esp32-idf3-20191209-v1.11-624-g210d05328.bin

esptool.py v2.8
Serial port com4
Connecting....
[Press the PRG button now if required]
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: cc:50:e3:80:1f:20
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Auto-detected Flash size: 4MB
Compressed 1240192 bytes to 783223...
Wrote 1240192 bytes (783223 compressed) at 0x00001000 in 69.7 seconds (effective 142.4 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [35]:
%serialconnect to --port COM4

Connecting to --port=COM4 --baud=115200 
MicroPython v1.11-624-g210d05328 on 2019-12-09; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [36]:
import os 
print(os.listdir())

['boot.py']


In [47]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-571-g7e374d231 on 2019-11-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x28edde56588, open=True>(port='COM4', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


## 無線網路設定

In [27]:
%serialconnect to --port COM4

Connecting to --port=COM4 --baud=115200 

I (455) cpu_start: Application information:
Connection successful
('192.168.43.150', '255.255.255.0', '192.168.43.254', '120.118.253.1')
[Timed out waiting for recognizable response]
Disconnecting [paste mode not working]
  (You may need to reset the device)

In [37]:
import network
import time

In [38]:
sta = network.WLAN(network.STA_IF)
print(sta.active(True))

True


In [39]:
nets = sta.scan()
print('SSID','\t\t\t\t','Channel','\t\t\t','dbm')
for net in nets:
    #print(net)
    ssid = net[0].decode('ascii')
    channel = net[2]
    dbm = net[3]
    print(ssid,'\t\t\t\t',channel,'\t\t\t',dbm)

SSID 				 Channel 			 dbm
informatics 				 7 			 -51
WAVELAN_CSU 				 11 			 -69
TANetRoaming 				 11 			 -69
802.1x_CSU 				 11 			 -71
802.1x_CSU 				 1 			 -74
WAVELAN_CSU 				 1 			 -75
TANetRoaming 				 1 			 -75
WAVELAN_CSU 				 1 			 -86
TANetRoaming 				 1 			 -88


In [40]:
SSID='informatics'
KEY='0953313123'
print(sta.active())
sta.connect(SSID,KEY)

True


In [41]:
import network
import ubinascii
mac = ubinascii.hexlify(sta.config('mac'),':').decode()
print(mac)
print(sta.ifconfig())

cc:50:e3:80:1f:20
('0.0.0.0', '0.0.0.0', '0.0.0.0', '0.0.0.0')


In [42]:
print(sta.isconnected())
for i in range(20):
    time.sleep(0.5)
    if sta.isconnected():
        break
print(sta.ifconfig())

True
('192.168.43.150', '255.255.255.0', '192.168.43.254', '120.118.253.1')


In [43]:
%sendtofile ap.txt
informatics
0953313123

Sent 2 lines (22 bytes) to ap.txt.


In [56]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.11-571-g7e374d231 on 2019-11-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.11-571-g7e374d231 on 2019-11-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [44]:
%sendtofile boot.py
# Complete project details at https://RandomNerdTutorials.com

import time
from umqttsimple import MQTTClient
import ubinascii
import machine
import micropython
import network
import esp
esp.osdebug(None)
import gc
gc.collect()

ssid = 'informatics'
password = '0953313123'
mqtt_server = 'gpssensor.ddns.net'
#EXAMPLE IP ADDRESS
#mqtt_server = '192.168.1.144'
client_id = ubinascii.hexlify(machine.unique_id())
topic_sub = b'/CSU/RT'
topic_pub = b'/CSU/T'
topic_sub1 = b'/CSU/RH'
topic_pub1 = b'/CSU/H'
topic_sub2 = b'/CSU/lamp'

last_message = 0
message_interval = 5
counter = 0

station = network.WLAN(network.STA_IF)

station.active(True)
station.connect(ssid, password)

while station.isconnected() == False:
  pass

print('Connection successful')
print(station.ifconfig())

Sent 39 lines (777 bytes) to boot.py.


In [45]:
%sendtofile main.py
# Complete project details at https://RandomNerdTutorials.com
from machine import Pin
import dht     
import time

p5=Pin(5, Pin.IN)
d=dht.DHT11(p5)        #建立 DHT11 物件
p2 = Pin(2, Pin.OUT)

def sub_cb(topic, msg):
  print((topic, msg))
  if topic == topic_sub:
    print('ESP received read T')
    T, H = read_TH()
    client.publish(topic_pub, str(T))
  elif topic == topic_sub1:
    T, H = read_TH()
    client.publish(topic_pub, str(H))
    print('ESP received read H')
  elif topic == topic_sub2 and msg == b'ON':
    p2.value(1)
    print('ESP received Lamp ON')
  elif topic == topic_sub2 and msg == b'OFF':
    p2.value(0)
    print('ESP received Lamp OFF')
  
  

def connect_and_subscribe():
  global client_id, mqtt_server, topic_sub
  client = MQTTClient(client_id, mqtt_server)
  client.set_callback(sub_cb)
  client.connect()
  client.subscribe(topic_sub)
  client.subscribe(topic_sub1)
  client.subscribe(topic_sub2)
  print('Connected to %s MQTT broker, subscribed to %s topic' % (mqtt_server, topic_sub))
  return client

def restart_and_reconnect():
  print('Failed to connect to MQTT broker. Reconnecting...')
  time.sleep(10)
  machine.reset()
  
def read_TH():
  print('Measuring....')
  d.measure()                  #重新測量溫溼度
  time.sleep(0.5)
  t=d.temperature()        #讀取攝氏溫度
  h=d.humidity()             #讀取相對溼度
  print('Temperature=', t, 'C', 'Humidity=', h, '%')
  return t,h

try:
  client = connect_and_subscribe()
except OSError as e:
  restart_and_reconnect()

p5=Pin(5, Pin.IN)
d=dht.DHT11(p5)        #建立 DHT11 物件

while True:
  try:
    client.check_msg()
    if (time.time() - last_message) > message_interval:
      T, H = read_TH()
      client.publish(topic_pub, str(T))
      client.publish(topic_pub1, str(H))
      last_message = time.time()
  except OSError as e:
    restart_and_reconnect()

Sent 71 lines (1832 bytes) to main.py.


In [46]:
%sendtofile umqttsimple.py
try:
    import usocket as socket
except:
    import socket
import ustruct as struct
from ubinascii import hexlify

class MQTTException(Exception):
    pass

class MQTTClient:

    def __init__(self, client_id, server, port=0, user=None, password=None, keepalive=0,
                 ssl=False, ssl_params={}):
        if port == 0:
            port = 8883 if ssl else 1883
        self.client_id = client_id
        self.sock = None
        self.server = server
        self.port = port
        self.ssl = ssl
        self.ssl_params = ssl_params
        self.pid = 0
        self.cb = None
        self.user = user
        self.pswd = password
        self.keepalive = keepalive
        self.lw_topic = None
        self.lw_msg = None
        self.lw_qos = 0
        self.lw_retain = False

    def _send_str(self, s):
        self.sock.write(struct.pack("!H", len(s)))
        self.sock.write(s)

    def _recv_len(self):
        n = 0
        sh = 0
        while 1:
            b = self.sock.read(1)[0]
            n |= (b & 0x7f) << sh
            if not b & 0x80:
                return n
            sh += 7

    def set_callback(self, f):
        self.cb = f

    def set_last_will(self, topic, msg, retain=False, qos=0):
        assert 0 <= qos <= 2
        assert topic
        self.lw_topic = topic
        self.lw_msg = msg
        self.lw_qos = qos
        self.lw_retain = retain

    def connect(self, clean_session=True):
        self.sock = socket.socket()
        addr = socket.getaddrinfo(self.server, self.port)[0][-1]
        self.sock.connect(addr)
        if self.ssl:
            import ussl
            self.sock = ussl.wrap_socket(self.sock, **self.ssl_params)
        premsg = bytearray(b"\x10\0\0\0\0\0")
        msg = bytearray(b"\x04MQTT\x04\x02\0\0")

        sz = 10 + 2 + len(self.client_id)
        msg[6] = clean_session << 1
        if self.user is not None:
            sz += 2 + len(self.user) + 2 + len(self.pswd)
            msg[6] |= 0xC0
        if self.keepalive:
            assert self.keepalive < 65536
            msg[7] |= self.keepalive >> 8
            msg[8] |= self.keepalive & 0x00FF
        if self.lw_topic:
            sz += 2 + len(self.lw_topic) + 2 + len(self.lw_msg)
            msg[6] |= 0x4 | (self.lw_qos & 0x1) << 3 | (self.lw_qos & 0x2) << 3
            msg[6] |= self.lw_retain << 5

        i = 1
        while sz > 0x7f:
            premsg[i] = (sz & 0x7f) | 0x80
            sz >>= 7
            i += 1
        premsg[i] = sz

        self.sock.write(premsg, i + 2)
        self.sock.write(msg)
        #print(hex(len(msg)), hexlify(msg, ":"))
        self._send_str(self.client_id)
        if self.lw_topic:
            self._send_str(self.lw_topic)
            self._send_str(self.lw_msg)
        if self.user is not None:
            self._send_str(self.user)
            self._send_str(self.pswd)
        resp = self.sock.read(4)
        assert resp[0] == 0x20 and resp[1] == 0x02
        if resp[3] != 0:
            raise MQTTException(resp[3])
        return resp[2] & 1

    def disconnect(self):
        self.sock.write(b"\xe0\0")
        self.sock.close()

    def ping(self):
        self.sock.write(b"\xc0\0")

    def publish(self, topic, msg, retain=False, qos=0):
        pkt = bytearray(b"\x30\0\0\0")
        pkt[0] |= qos << 1 | retain
        sz = 2 + len(topic) + len(msg)
        if qos > 0:
            sz += 2
        assert sz < 2097152
        i = 1
        while sz > 0x7f:
            pkt[i] = (sz & 0x7f) | 0x80
            sz >>= 7
            i += 1
        pkt[i] = sz
        #print(hex(len(pkt)), hexlify(pkt, ":"))
        self.sock.write(pkt, i + 1)
        self._send_str(topic)
        if qos > 0:
            self.pid += 1
            pid = self.pid
            struct.pack_into("!H", pkt, 0, pid)
            self.sock.write(pkt, 2)
        self.sock.write(msg)
        if qos == 1:
            while 1:
                op = self.wait_msg()
                if op == 0x40:
                    sz = self.sock.read(1)
                    assert sz == b"\x02"
                    rcv_pid = self.sock.read(2)
                    rcv_pid = rcv_pid[0] << 8 | rcv_pid[1]
                    if pid == rcv_pid:
                        return
        elif qos == 2:
            assert 0

    def subscribe(self, topic, qos=0):
        assert self.cb is not None, "Subscribe callback is not set"
        pkt = bytearray(b"\x82\0\0\0")
        self.pid += 1
        struct.pack_into("!BH", pkt, 1, 2 + 2 + len(topic) + 1, self.pid)
        #print(hex(len(pkt)), hexlify(pkt, ":"))
        self.sock.write(pkt)
        self._send_str(topic)
        self.sock.write(qos.to_bytes(1, "little"))
        while 1:
            op = self.wait_msg()
            if op == 0x90:
                resp = self.sock.read(4)
                #print(resp)
                assert resp[1] == pkt[2] and resp[2] == pkt[3]
                if resp[3] == 0x80:
                    raise MQTTException(resp[3])
                return

    # Wait for a single incoming MQTT message and process it.
    # Subscribed messages are delivered to a callback previously
    # set by .set_callback() method. Other (internal) MQTT
    # messages processed internally.
    def wait_msg(self):
        res = self.sock.read(1)
        self.sock.setblocking(True)
        if res is None:
            return None
        if res == b"":
            raise OSError(-1)
        if res == b"\xd0":  # PINGRESP
            sz = self.sock.read(1)[0]
            assert sz == 0
            return None
        op = res[0]
        if op & 0xf0 != 0x30:
            return op
        sz = self._recv_len()
        topic_len = self.sock.read(2)
        topic_len = (topic_len[0] << 8) | topic_len[1]
        topic = self.sock.read(topic_len)
        sz -= topic_len + 2
        if op & 6:
            pid = self.sock.read(2)
            pid = pid[0] << 8 | pid[1]
            sz -= 2
        msg = self.sock.read(sz)
        self.cb(topic, msg)
        if op & 6 == 2:
            pkt = bytearray(b"\x40\x02\0\0")
            struct.pack_into("!H", pkt, 2, pid)
            self.sock.write(pkt)
        elif op & 6 == 4:
            assert 0

    # Checks whether a pending message from server is available.
    # If not, returns immediately with None. Otherwise, does
    # the same processing as wait_msg.
    def check_msg(self):
        self.sock.setblocking(False)
        return self.wait_msg()

Sent 207 lines (6445 bytes) to umqttsimple.py.


In [47]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-624-g210d05328 on 2019-12-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x1e87a935e88, open=True>(port='COM4', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
